In [71]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import os
import cv2
import pytesseract
import base64
import numpy as np
import win32gui
import win32con
import re

options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")
options.add_argument("--disable-notifications")
options.add_argument("--ignore-certificate-errors")
options.add_argument("--allow-running-insecure-content")
options.add_argument("--no-default-browser-check")
options.add_argument("--disable-infobars")

exp_opt = ["enable-logging", "enable-automation", "ignore-certificate-errors"]
options.add_experimental_option("excludeSwitches", exp_opt)
#options.add_argument(r"--user-data-dir=C:\Users\Luciano Carvajal\AppData\Local\Google\Chrome\User Data")


def opendriver():
    driver = webdriver.Chrome(options=options)
    return driver

In [72]:
def create_folders(keyword):
    print("\n\nRevisando existencia de carpetas necesarias")
    dir_path = os.path.dirname(os.path.realpath("__file__"))
    print("path local: ", dir_path)
    folderpath = "Getonboard"
    folderpath = os.path.join(dir_path, folderpath)
    if not os.path.exists(folderpath):
        os.mkdir(folderpath)
        print("Carpeta Getonboard creada")
    else: print("Carpeta Getonboard existente")
    dir_path = folderpath
    #folderpath = keyword.replace(" ", "_") + "_" + str(datetime.now().date())
    folderpath = os.path.join(dir_path, "Data")
    if not os.path.exists(folderpath):
        os.mkdir(folderpath)
        print("Subcarpeta Data creada\n\n")
    else: print("Subcarpeta Data Existente")
    return folderpath

In [73]:
def login(driver, mail, clave):
    userdiv = driver.find_element(By.XPATH,'//*[@id="username"]')
    userdiv.click()
    userdiv.clear()
    userdiv.send_keys(mail)

    passdiv = driver.find_element(By.XPATH,'//*[@id="password"]')
    passdiv.click()
    passdiv.clear()
    passdiv.send_keys(clave)

    driver.find_element(By.XPATH,'//*[@type="submit"]').click()
    time.sleep(1)
    print("Cuenta Logueada\n")
    time.sleep(2)

In [74]:
def obtain_date(fecha_str):

    # Diccionario para mapear los nombres de los meses en español a números
    meses_espanol = {
        "enero": 1,
        "febrero": 2,
        "marzo": 3,
        "abril": 4,
        "mayo": 5,
        "junio": 6,
        "julio": 7,
        "agosto": 8,
        "septiembre": 9,
        "octubre": 10,
        "noviembre": 11,
        "diciembre": 12
    }

    # Separar el string en día, mes y año
    dia_str, mes_str, anio_str = fecha_str.split(" de ")

    # Convertir el día y el año a enteros
    dia = int(dia_str)
    anio = int(anio_str)

    # Obtener el número del mes a partir del diccionario
    mes = meses_espanol[mes_str.lower()]

    # Crear un objeto datetime con la fecha
    fecha_estandar = datetime(anio, mes, dia)

    # Formatear la fecha como string en formato estándar (yyyy-mm-dd)
    fecha_estandar_str = fecha_estandar.strftime("%Y-%m-%d")
    return fecha_estandar_str

In [75]:
driver = opendriver()
url = "https://www.getonbrd.com/auth/linkedin"
driver.get(url)


In [76]:
clave = "polghtherce1"
mail = "laserena.arbolada@gmail.com"
login(driver, mail, clave)

Cuenta Logueada



In [77]:
#driver.find_element(By.XPATH,'//*[@id="accept_cookies"]').click()
#driver.find_element(By.XPATH, '//div[@class="dropdown gb-login ml-3"]/button').click()
#driver.find_element(By.XPATH, '//div[@class="dropdown gb-login ml-3 open"]/ul/li[2]/a').click()
#driver.find_element(By.XPATH, '//a[contains(text(),"con tu email")]').click()
### Ya abierta la opción
#mail = "lcarvajalrojas3@gmail.com"
#userdiv = driver.find_element(By.XPATH, '//input[@name="email"]')
#userdiv.click()
#userdiv.clear()
#userdiv.send_keys(mail)
#driver.find_element(By.XPATH, '//button[contains(text(),"Continuar")]').click()

In [88]:
list_of_links = driver.find_elements(By.XPATH,'//*[@id="jobs-result"]/div/a')
### Tras obtenida la lista de links, se podría revisar cuáles han sido revisados anteriormente, descartar y luego revisar
## La carga de links tiene un límite, que es alrededor de 500 (donde se acaban los avisos activos filtrados), allí es 
## donde se puede scrollear, recolectar y comenzar el scraping
while True:
    try: 
        button = driver.find_element(By.XPATH, "//button[contains(text(), 'Cargar más empleos')]")
        button.click()
        print("Cargando avisos")
        time.sleep(5)
    except:
        print("Boton de carga no encontrado / Carga completa de avisos")
        break

Cargando avisos
Cargando avisos
Cargando avisos
Boton de carga no encontrado / Carga completa de avisos


In [90]:
list_of_ads = driver.find_elements(By.XPATH,'//*[@id="jobs-result"]/div/a')
list_of_links = []
for link in list_of_ads:
    list_of_links.append(link.get_attribute("href"))
len(list_of_links)

['https://www.getonbrd.com/empleos/operaciones-management/jefe-de-proyectos-ti-tcit-santiago-1bd5', 'https://www.getonbrd.com/empleos/cybersecurity/auditor-de-sistemas-senior-empresas-sb-remote', 'https://www.getonbrd.com/empleos/comercial-ventas/saas-sales-closer-sales-kiwi-remote', 'https://www.getonbrd.com/empleos/operaciones-management/technical-project-manager-devsu-remote-315a', 'https://www.getonbrd.com/empleos/comercial-ventas/business-development-representative-devsu-remote', 'https://www.getonbrd.com/empleos/comercial-ventas/sales-development-representative-rankmi-santiago', 'https://www.getonbrd.com/empleos/comercial-ventas/sales-development-representative-chile-enviame-remote', 'https://www.getonbrd.com/empleos/comercial-ventas/lead-generation-specialist-sales-kiwi-remote-1607', 'https://www.getonbrd.com/empleos/comercial-ventas/sales-closer-sales-kiwi-remote-605d', 'https://www.getonbrd.com/empleos/comercial-ventas/sales-development-representative-sdr-healthatom-bogota', '

In [91]:
driver.get(list_of_links[0])

In [211]:
#### SCRAP
def scrap_add():
    title = driver.find_element(By.XPATH,'//span[@itemprop="title"]').text
    print("Título: ", title)

    codigo_a_nombre = {
    'cl': 'Chile',
    'pe': 'Perú',
    'ar': 'Argentina',
    'us': 'Estados Unidos',
    'co': 'Colombia',
    'ec': 'Ecuador',
    'mx': 'México'
    }

    try:
        pais = driver.find_element(By.XPATH,'//span[@class="location-flag"]').get_attribute("style")
        pattern = r'\/([a-z]{2})_mini\.png'
        match = re.search(pattern, pais)
        pais = match.group(1)
        pais = codigo_a_nombre.get(pais, pais)
        print("País:", pais)
    except: 
        pais = ""
        print("País:", pais)

    pattern = r"^(.*?)\s\((Presencial|Híbrido)\)$"
    full_address = driver.find_element(By.XPATH,'//span[@itemprop="address"]').text.strip()
    resultado = re.search(pattern, full_address)
    if resultado:
        city = resultado.group(1).strip()
        job_type = resultado.group(2)
        print("Nombre de la ciudad:", city)
        print("Tipo de trabajo:", job_type)
    else:
        pattern = r"^\s*Remoto \((.*?)\)$"
        resultado = re.search(pattern, full_address)
        city = ""
        job_type = full_address.split(" ")[0]
        print("Tipo de trabajo:", job_type)
        if resultado:
            pais = resultado.group(1).strip()
            print("Pais:", pais)
            
        else:
            print("No se encontró el patrón de país en el texto.")

    qualifications = driver.find_element(By.XPATH, "//span[@itemprop='qualifications']").text.strip()
    print("Nivel:", qualifications)

    mode = driver.find_element(By.XPATH, "//span[@itemprop='employmentType']")
    mode = driver.execute_script("return arguments[0].textContent;", mode)

    print("Modalidad:",mode)

    area = driver.find_element(By.XPATH, "//h2/a").text.strip()
    print("Area:", area)

    try:
        salary = driver.find_element(By.XPATH, "//span[@itemprop='baseSalary']/span/strong").text.strip()
        salary_str = salary.replace("$", "")  # Eliminar el símbolo de dólar
        try:
            min_salary, max_salary = map(int, salary_str.split("-"))
            salary = (min_salary + max_salary) / 2
            print("Salario:", salary)
        except:
            salary = salary_str.replace(",", "")
            print("Salario:", salary)
    except:
        salary = "No hay información"
        print("Salario:", salary)

    publishdate = driver.find_element(By.XPATH, "//div[@itemprop='hiringOrganization']//time").text
    publishdate = obtain_date(publishdate)
    print("Fecha de publicación:", publishdate)

    applicants = driver.find_element(By.XPATH, "//div[@class='size0 mt1']").text.split(" ")
    print("Postulaciones:", applicants[0])

    companyname = driver.find_element(By.XPATH, "//div[@itemprop='hiringOrganization']//strong")
    companyname = driver.execute_script("return arguments[0].textContent;", companyname)
    print("Empresa:", companyname)

    skills_list = driver.find_elements(By.XPATH, "//div[@itemprop='skills']/a")
    skills = []
    for sk in skills_list:
        skills.append(sk.text)
    print(skills)

    h3_texts = [
    'Job functions',
    'Funciones del cargo',
    'Qué harás?'
    ]

    for h3_text in h3_texts:
        try:
            xpath_expression = f"//div[@itemprop='description']/div/div/div[contains(h3, '{h3_text}')]/div"
            job_functions = driver.find_element(By.XPATH, xpath_expression).text
            break
        except:
            job_functions = ""
    print("Funciones", job_functions)

    h3_texts = [
    'Qualifications and requirements',
    'Requerimientos del cargo',
    '¿Qué necesitamos?',
    '¿Qué buscamos?',
    '¿Qué esperamos de ti?',
    '¿Cómo nos gustaría que fueras?',
    'Debes postular a este rol solo si:'
    
    ]

    for h3_text in h3_texts:
        try:
            xpath_expression = f"//div[@itemprop='description']/div/div/div[contains(h3, '{h3_text}')]/div"
            job_requirements = driver.find_element(By.XPATH, xpath_expression).text
            break  # Salir del ciclo si se encontró el elemento
        except:
            job_requirements = ""

    #xpath_expression = "//div[@itemprop='description']/div/div/div[contains(h3, 'Qualifications and requirements') or contains(h3, 'Requerimientos del cargo') or contains(h3, '¿Qué esperamos de ti?') or contains (h3, '¿Cómo nos gustaría que fueras?') or contains(h3, 'Debes postular a este rol solo si:')]/div"
    #job_requirements = driver.find_element(By.XPATH, xpath_expression)
    print(job_requirements)

In [213]:
#driver.get(list_of_links[37])
scrap_add()

Título:  Lead Data Scientist
País: 
Tipo de trabajo: Remoto
No se encontró el patrón de país en el texto.
Nivel: Senior
Modalidad: FULL_TIME
Area: Data Science / Analytics
Salario: 2400.0
Fecha de publicación: 2023-07-21
Postulaciones: 48
Empresa: Myfuture-AI
['MySQL', 'Python', 'Data Science', 'PyTorch', 'Back-end', 'Machine Learning', 'Machine Learning Engineer', 'Linux']
Funciones 

